In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%gui qt5

In [3]:
import napari

In [4]:
import pickle
import itertools
import numpy as np
from pathlib import Path
from skimage import measure
from dask.distributed import Client

In [5]:
from pysmFISH.stitching import organize_square_tiles
from pysmFISH.data_models import Dataset
from pysmFISH.processing_cluster_setup import local_cluster_setup

/Users/simone.codeluppi/opt/miniconda3/envs/test_d/lib/python3.8/site-packages/dask_jobqueue/core.py:19: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/Users/simone.codeluppi/opt/miniconda3/envs/test_d/lib/python3.8/site-packages/dask_jobqueue/core.py:19: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/Users/simone.codeluppi/opt/miniconda3/envs/test_d/lib/python3.8/site-packages/dask_jobqueue/htcondor.py:6: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import parse_bytes


In [6]:
def stitch_using_coords_general_segmented_objects(fov,obj_dict,tile_corners_coords_pxl,reference_corner_fov_position, metadata):
    """
    Tiles are placed directly on the position indicated by the microscope
    coords
    """

    r_microscope_coords = tile_corners_coords_pxl[fov,0]
    c_microscope_coords = tile_corners_coords_pxl[fov,1]
    
    if obj_dict:
        
        if reference_corner_fov_position == 'top-left':
            for el,coords_dict in obj_dict.items():
                coords_dict['stitched_coords'] = np.vstack([r_microscope_coords + coords_dict['original_coords'][:,0],
                                                            c_microscope_coords + coords_dict['original_coords'][:,1]]).T

        elif reference_corner_fov_position == 'top-right':
            for el,coords_dict in obj_dict.items():
                coords_dict['stitched_coords'] = np.vstack([r_microscope_coords + coords_dict['original_coords'][:,0],
                                                            c_microscope_coords - (metadata['img_width'] -coords_dict['original_coords'][:,1])]).T

        elif reference_corner_fov_position == 'bottom_left':
            for el,coords_dict in obj_dict.items():
                coords_dict['stitched_coords'] = np.vstack([r_microscope_coords + (metadata['img_height'] -coords_dict['original_coords'][:,0]),
                                                            c_microscope_coords + coords_dict['original_coords'][:,1]]).T

    return obj_dict

In [7]:
def register_coords_obj(fov,segmentation_output_path):
    segmented_output = pickle.load(open(segmentation_output_path / ('preprocessed_data_fov_' + str(fov) + '_mask.pkl'), 'rb'))
    segmented_regions = measure.regionprops(segmented_output)
    segmented_regions_dict = {}
    for prop in segmented_regions:
        segmented_regions_dict[str(fov)+'-'+str(prop.label)] = {}
        segmented_regions_dict[str(fov)+'-'+str(prop.label)]['original_coords']=prop.coords
        segmented_regions_dict[str(fov)+'-'+str(prop.label)]['stitched_coords']= np.nan
        segmented_regions_dict = stitch_using_coords_general_segmented_objects(fov,segmented_regions_dict,
                                                                         stitching_parameters,reference_corner_fov_position, metadata)
        pickle.dump(segmented_regions_dict,open(segmentation_output_path / ('registered_objs_dict_fov_' + str(fov) + '.pkl'), 'wb'))

In [8]:

def load_stitched_segmented_data(fov,segmentation_output_path):
    segmented_region_img = pickle.load(open(segmentation_output_path / ('registered_objs_dict_fov_' + str(fov) + '.pkl'), 'rb'))

    obj_segmented_img = {el:coords_dict['stitched_coords'] for (el,coords_dict) in segmented_region_img.items()}
    return obj_segmented_img

def objects_overlapping_region(obj_segmented_img,chunk_coords):
    objs_list=list()
    for lab in obj_segmented_img.keys():
        if ((obj_segmented_img[lab][:,0] >= chunk_coords[0]).any() and
            (obj_segmented_img[lab][:,0] <= chunk_coords[1]).any() and
            (obj_segmented_img[lab][:,1] >= chunk_coords[2]).any() and
            (obj_segmented_img[lab][:,1] <= chunk_coords[3]).any()):
                objs_list.append(lab)
    return objs_list

def overlapping_objs(obj_segmented_img1,obj_segmented_img2,chunk_coords,min_overlapping_pixels_segmentation = 20):
    objs_list_img1 = objects_overlapping_region(obj_segmented_img1,chunk_coords)
    objs_list_img2 = objects_overlapping_region(obj_segmented_img2,chunk_coords)
    
    all_combinations = list(itertools.product(objs_list_img1,objs_list_img2))
    
    intersection = []
    overlapping_sizes = {}
    for couple in all_combinations:
        obj1_set = set(map(tuple, obj_segmented_img1[couple[0]].astype(int)))
        obj2_set = set(map(tuple, obj_segmented_img2[couple[1]].astype(int)))
        result=not obj1_set.isdisjoint(obj2_set)
        if result:
            # Check minimum number of pixels to correct for potential error caused by the rounding
            number_overlapping_pixels = len(obj1_set.intersection(obj2_set))
            if number_overlapping_pixels >= min_overlapping_pixels_segmentation:
                intersection.append(result)
                overlapping_sizes[couple] = number_overlapping_pixels
            else:
                intersection.append(not result)
        else:
            intersection.append(result)
    if (np.array(intersection)).any():
        all_combinations = np.array(all_combinations)
        intersecting_cpls = all_combinations[intersection]
    else:
        intersecting_cpls = []
    return intersecting_cpls, overlapping_sizes



def correct_multiple_overlapping_objects(intersecting_cpls,overlapping_sizes):

    intersecting_cpls_tpl_trimmed = []
    intersecting_cpls_tpl = [tuple(x) for x in intersecting_cpls]
    
    intersecting_cpls_removed_overlapping = intersecting_cpls_tpl.copy()
    tracker = intersecting_cpls_tpl.copy()

    combined_intersecting_cpls_tpl = []
    tracker = []
    reducer = intersecting_cpls_tpl.copy()
    for tpl in intersecting_cpls_tpl:
        if tpl not in tracker:
            combined = [tpl]
            tracker.append(tpl)
            reducer.remove(tpl)
            runner = reducer
            for cpl in runner:
                if set(tpl).intersection(cpl):
                    tracker.append(cpl)
                    combined.append(cpl)
                    reducer.remove(cpl)
            combined_intersecting_cpls_tpl.append(combined)

    #combined_intersecting_cpls_tpl = set(combined_intersecting_cpls_tpl)
    for grp in combined_intersecting_cpls_tpl:
        if len(grp) >1 :
            # Select the object with largest overlap (maybe not best criteria)
            selected = { your_key: overlapping_sizes[your_key] for your_key in grp }
            counts = list(selected.values())
            keys = list(selected.keys())
            selected_cpl = keys[counts.index(max(counts))]
            intersecting_cpls_tpl_trimmed.append(selected_cpl)
        else:
            intersecting_cpls_tpl_trimmed.append(grp)
    
    return intersecting_cpls_removed_overlapping,overlapping_sizes



def merge_objects(obj_segmented_img1,obj_segmented_img2,intersecting_cpls):
    merged_objs = {}
    for cpl in intersecting_cpls:
        obj1 = obj_segmented_img1[cpl[0]].astype(int)
        obj2 = obj_segmented_img2[cpl[1]].astype(int)
        merged_obj = np.vstack((obj1,obj2))
        merged_obj = np.unique(merged_obj,axis=0)
        merged_objs[cpl[0]] = merged_obj
    return merged_objs



In [9]:
def adjust_overlapping_objects(obj_segmented_img1,obj_segmented_img2,chunk_coords,segmentation_output_path,min_overlapping_pixels_segmentation = 20):
    
    intersecting_cpls, overlapping_sizes = overlapping_objs(obj_segmented_img1,obj_segmented_img2,chunk_coords,min_overlapping_pixels_segmentation)
    
    # Correct for multiple overlapping large objects
    intersecting_cpls,overlapping_sizes = correct_multiple_overlapping_objects(intersecting_cpls,overlapping_sizes)
    
    objs = merge_objects(obj_segmented_img1,obj_segmented_img2,intersecting_cpls)
    
    #for cpl in intersecting_cpls:
     #   del obj_segmented_img1[cpl[0]]
     #   del obj_segmented_img2[cpl[1]]
     #   obj_segmented_img1[cpl[0]] = objs[cpl[0]]
    
    return intersecting_cpls, objs
#obj_segmented_img1, obj_segmented_img2

In [10]:

def remove_overlapping_obj(fov,
                           nuclei_org_tiles,
                          segmentation_output_path,
                          min_overlapping_pixels_segmentation = 20):

    overlapping_dict = nuclei_org_tiles.overlapping_regions[fov]
    all_obj_segmented = {}
    output_cpl = {}
    corner_cpl = []
    
    
    # Correct overlapping objects at the intersection between the images
    for cpl, chunk_coords in overlapping_dict.items():
        obj_segmented_img1 = load_stitched_segmented_data(cpl[0],segmentation_output_path)
        obj_segmented_img2 = load_stitched_segmented_data(cpl[1],segmentation_output_path)
        intersecting_cpls, objs = adjust_overlapping_objects(obj_segmented_img1,obj_segmented_img2,chunk_coords,segmentation_output_path,min_overlapping_pixels_segmentation)
        all_obj_segmented[cpl[0]] = obj_segmented_img1
        all_obj_segmented[cpl[1]] = obj_segmented_img2
        output_cpl[cpl] = (intersecting_cpls, objs)
        corner_cpl.append(cpl[1])

        
    # Check if processing fov has one single neighbour
    if len(overlapping_dict.keys()) == 2:
        # Correct overlapping objects at the corner
        intersecting_cpls, objs = adjust_overlapping_objects(all_obj_segmented[corner_cpl[0]],all_obj_segmented[corner_cpl[1]],chunk_coords,segmentation_output_path,min_overlapping_pixels_segmentation)
        all_obj_segmented[corner_cpl[0]] = obj_segmented_img1
        all_obj_segmented[corner_cpl[1]] = obj_segmented_img2
        output_cpl[tuple(corner_cpl)] = (intersecting_cpls, objs)
    
    
    # Format the output
    all_cpls = []
    all_intersecting_step = []
    all_obj_step = {}
    for cpl, (intersecting_cpls, objs) in output_cpl.items():
        all_intersecting_step.append(intersecting_cpls)
        all_obj_step.update(objs)
        all_cpls.append(cpl)

    unraveled_intersecting_step = [val for big_list in all_intersecting_step for el in big_list for val in el ]
    
    # To use in case to sort by fov for parallelisation
    #all_couples = set([el for cpl in all_cpls for el in cpl])
    #for el in all_couples:
     #   intersecting_step_dict[el] = [val for val in unraveled_intersecting_step if str(el) in val]

    
    return unraveled_intersecting_step,all_obj_step
    
    

In [11]:
client = Client()

In [12]:
cores = 4
memory = 10
cluster = local_cluster_setup(cores=cores,memory=memory)
client = Client(cluster)

/Users/simone.codeluppi/opt/miniconda3/envs/test_d/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 62774 instead
  warnings.warn(


In [13]:
client

distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distribu

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:62774/status,
Dashboard: http://127.0.0.1:62774/status,Workers: 0
Total threads: 0,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62775,Workers: 0
Dashboard: http://127.0.0.1:62774/status,Total threads: 0
Started: Just now,Total memory: 0 B


distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distribu

# Prepare processing parameters

In [23]:
experiment_fpath = Path('/Users/simone.codeluppi/Documents/data_analysis_jlabs_sc/notebooks_pysmFISH/processing_folder/LBEXP20210718_EEL_Mouse_448_2/')
segmentation_output_path = experiment_fpath / 'fresh_tissue' / 'Cellpose_segmentation'

In [24]:
#experiment_fpath = Path('processing_folder/LBEXP20210718_EEL_Mouse_448_2')
fresh_tissue_path = experiment_fpath / 'fresh_tissue'
dataset_name = '211014_09_56_53_20210718_124116_843__ChannelCy3_Nuclei_Seq0001_img_data_dataset.parquet'
dataset_name = '211014_09_56_52_20210718_124116_843__ChannelEuropium_Cy3_Seq0002_img_data_dataset.parquet'
round_num = 1
overlapping_percentage = 5

In [25]:
nuclei_data = Dataset()
nuclei_data.load_dataset(fresh_tissue_path / dataset_name)
nuclei_data.dataset['processing_type'] = 'undefined'
nuclei_data.dataset['overlapping_percentage'] = 5 / 100
nuclei_data.dataset['machine'] = 'ROBOFISH2'

metadata = nuclei_data.collect_metadata(nuclei_data.dataset)

In [ ]:
stitching_parameters = pickle.load(open(experiment_fpath / 'fresh_tissue' / 'results' / 'global_stitched_coords.pkl','rb') )

In [15]:
nuclei_org_tiles = organize_square_tiles(experiment_fpath,nuclei_data.dataset,metadata,round_num)
nuclei_org_tiles.run_tiles_organization()

In [16]:
nuclei_org_tiles.tile_corners_coords_pxl = stitching_parameters
nuclei_org_tiles.determine_overlapping_regions()
reference_corner_fov_position = nuclei_org_tiles.reference_corner_fov_position

In [17]:
reference_corner_fov_position

'top-right'

# Register the segmented objects to the stitched coords

In [19]:
all_futures = []
for fov in nuclei_data.dataset.fov_num.unique():
    future = client.submit(register_coords_obj,fov,segmentation_output_path)
    all_futures.append(future)
_ = client.gather(all_futures)

# Remove the overlapping objects

In [22]:
min_overlapping_pixels_segmentation = 20
all_futures = []
for fov in nuclei_data.dataset.fov_num.unique():
    future = client.submit(remove_overlapping_obj,
                           fov,
                           nuclei_org_tiles,
                          segmentation_output_path,
                          min_overlapping_pixels_segmentation)
    all_futures.append(future)
all_outputs = client.gather(all_futures)


KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [561]:
# tmp_saving
pickle.dump(all_outputs,open(segmentation_output_path / ('output_removal_tmp.pkl'), 'wb'))

In [26]:

all_outputs = pickle.load(open(segmentation_output_path / ('output_removal_tmp.pkl'), 'rb'))

In [27]:
all_obj_original = {}
for fov in nuclei_data.dataset.fov_num.unique():
    segmented_region_img = load_stitched_segmented_data(fov,segmentation_output_path)
    all_obj_original.update(segmented_region_img)

In [28]:
all_remove_objs_list = []
all_add_objs = {}
for rem, obj in all_outputs:
    all_remove_objs_list.append(rem)
    all_add_objs.update(obj)
all_remove_objs_list = [val for el in all_remove_objs_list for val in el ]

In [29]:
for obj in all_remove_objs_list:
    all_obj_original.pop(obj,None)

In [30]:
all_obj_original.update(all_add_objs)

In [32]:
pickle.dump(all_obj_original,open(segmentation_output_path / 'all_obj_output_dict.pkl','wb'))

In [23]:
# Calculate image size 

global_stitched_coords = pickle.load(open(experiment_fpath / 'fresh_tissue' / 'results' / 'global_stitched_coords.pkl','rb'))

row_min = global_stitched_coords[:,0].min()
col_min = global_stitched_coords[:,1].min()
row_max = global_stitched_coords[:,0].max()
col_max = global_stitched_coords[:,1].max()

nrows = np.int(row_max - row_min + 2048)
ncols = np.int(col_max - col_min + 2048)

img = np.zeros([nrows,ncols])

In [24]:
for idx, (tag, coords) in enumerate(all_obj_original.items()):
    coords = coords -[row_min,col_min]
    coords = coords.astype(int)
    img[coords[:,0],coords[:,1]] = idx

In [26]:
np.save(segmentation_output_path / 'stitched_img_tmp.npy', img)

In [34]:
nrows

29287

In [35]:
ncols

60412

In [43]:
len(all_obj_original.keys())

145573

In [42]:
vw = napari.Viewer()
_ = vw.add_labels(img)

In [32]:
from skimage import io

In [33]:
io.imsave('~/Downloads/stitched_40X.png',img)

Lossy conversion from float64 to uint8. Range [0.0, 145572.0]. Convert image to uint8 prior to saving to suppress this warning.


In [548]:
remove_objs_dict = {}
add_objs_dict = {}
add_objs_dict_combined = {}
for remove_single in all_remove_objs_list:
    for key, items in remove_single.items():
        if key not in remove_objs_dict.keys():
            remove_objs_dict[key] = []
        remove_objs_dict[key].append(items)
        
for add_single in all_add_objs_list:
    if isinstance(add_single,dict):
        add_objs_dict.update(add_single)
        for key, items in add_objs_dict.items():
            if key not in add_objs_dict_combined.keys():
                add_objs_dict_combined[key] = []
            add_objs_dict_combined[key].append(items)
         
        
        

14777

In [537]:
def update(fov,
                           nuclei_org_tiles,
                          segmentation_output_path,
                          min_overlapping_pixels_segmentation = 20):

    overlapping_dict = nuclei_org_tiles.overlapping_regions[fov]
    all_obj_segmented = {}
    output_cpl = {}
    corner_cpl = []
    
    
    # Correct overlapping objects at the intersection between the images
    for cpl, chunk_coords in overlapping_dict.items():
        obj_segmented_img1, obj_segmented_img2 = load_segmented_data(cpl,segmentation_output_path)
        intersecting_cpls, objs = adjust_overlapping_objects(obj_segmented_img1,obj_segmented_img2,chunk_coords,segmentation_output_path,min_overlapping_pixels_segmentation)
        all_obj_segmented[cpl[0]] = obj_segmented_img1
        all_obj_segmented[cpl[1]] = obj_segmented_img2
        output_cpl[cpl] = (intersecting_cpls, objs)
        corner_cpl.append(cpl[1])

        
    # Check if processing fov has one single neighbour
    if len(overlapping_dict.keys()) == 2:
        # Correct overlapping objects at the corner
        intersecting_cpls, objs = adjust_overlapping_objects(all_obj_segmented[corner_cpl[0]],all_obj_segmented[corner_cpl[1]],chunk_coords,segmentation_output_path,min_overlapping_pixels_segmentation)
        all_obj_segmented[corner_cpl[0]] = obj_segmented_img1
        all_obj_segmented[corner_cpl[1]] = obj_segmented_img2
        output_cpl[tuple(corner_cpl)] = (intersecting_cpls, objs)
    
    
    # Format the output
    all_cpls = []
    all_intersecting_step = []
    all_obj_step = {}
    for cpl, (intersecting_cpls, objs) in output_cpl.items():
        all_intersecting_step.append(intersecting_cpls)
        all_obj_step.update(objs)
        all_cpls.append(cpl)

    unraveled_intersecting_step = [val for big_list in all_intersecting_step for el in big_list for val in el ]
    all_couples = set([el for cpl in all_cpls for el in cpl])
    for el in all_couples:
        intersecting_step_dict[el] = [val for val in unraveled_intersecting_step if str(el) in val]

    
    return intersecting_step_dict,all_obj_step
    

305

In [508]:
output_step = all_outputs[100]
all_cpls = []
all_intersecting_step = []
all_obj_step = {}
for cpl, (intersecting_cpls, objs) in output_step.items():
    all_intersecting_step.append(intersecting_cpls)
    all_obj_step.update(objs)
    all_cpls.append(cpl)
    
unraveled_intersecting_step = [val for big_list in all_intersecting_step for el in big_list for val in el ]
all_couples = set([el for cpl in all_cpls for el in cpl])
for el in all_couples:
    intersecting_step_dict[el] = [val for val in unraveled_intersecting_step if str(el) in val]


In [511]:
unraveled_intersecting_step = [val for big_list in all_intersecting_step for el in big_list for val in el ]
all_couples = set([el for cpl in all_cpls for el in cpl])

In [519]:
intersecting_step_dict = {}
for el in all_couples:
    intersecting_step_dict[el] = [val for val in unraveled_intersecting_step if str(el) in val]
    

In [520]:
intersecting_step_dict

{225: ['225-533',
  '225-529',
  '225-534',
  '225-530',
  '225-532',
  '225-531',
  '225-535',
  '225-533',
  '225-537',
  '225-536',
  '225-540',
  '225-539',
  '225-538',
  '225-541',
  '225-542',
  '225-543',
  '225-544',
  '225-545',
  '225-546',
  '225-547',
  '225-548',
  '225-549',
  '225-550',
  '225-551'],
 189: ['189-29',
  '189-48',
  '189-85',
  '189-127',
  '189-147',
  '189-157',
  '189-165',
  '189-191',
  '189-244',
  '189-260',
  '189-262',
  '189-269',
  '189-289',
  '189-301',
  '189-305',
  '189-305',
  '189-308',
  '189-322',
  '189-346',
  '189-370',
  '189-370',
  '189-373',
  '189-384',
  '189-401',
  '189-409',
  '189-439',
  '189-445',
  '189-455',
  '189-462',
  '189-467',
  '189-474',
  '189-483',
  '189-486',
  '189-513',
  '189-521',
  '189-524',
  '189-524',
  '189-532',
  '189-3',
  '189-4',
  '189-6',
  '189-7',
  '189-8',
  '189-9',
  '189-10',
  '189-11',
  '189-11',
  '189-12',
  '189-13',
  '189-14',
  '189-15',
  '189-16',
  '189-17',
  '189-18',


In [463]:
fov = 10
overlapping_dict = nuclei_org_tiles.overlapping_regions[fov]

    

In [464]:
overlapping_dict

{(10, 23): [25290.464561575867,
  25393.297674641493,
  31031.28497439398,
  33079.28497439398],
 (10, 11): [25290.464561575867,
  27338.464561575867,
  32974.81797593158,
  33079.28497439398]}

In [381]:
intersecting_cpls_tpl = [('121-34', '174-2019'),
 ('121-34', '174-2022'),
 ('121-35', '174-2024'),
 ('121-35', '174-2025'),
 ('121-36', '174-2020'),
 ('121-36', '174-2027'),
 ('121-38', '174-2023'),
 ('121-39', '174-2024')]

In [482]:
overlapping_dict = nuclei_org_tiles.overlapping_regions[fov]
all_obj_segmented = {}
output_cpl = {}
corner_cpl = []
    
    
fov = 10
overlapping_dict = nuclei_org_tiles.overlapping_regions[fov]

cpl = (10,23)
chunk_coords = overlapping_dict[cpl]
obj_segmented_img1, obj_segmented_img2 = load_segmented_data(cpl,segmentation_output_path)
#intersecting_cpls, overlapping_sizes = overlapping_objs(obj_segmented_img1,obj_segmented_img2,chunk_coords,min_overlapping_pixels_segmentation)


In [475]:
intersecting_cpls,overlapping_sizes = correct_multiple_overlapping_objects(intersecting_cpls,overlapping_sizes)

In [484]:
intersecting_cpls, objs = adjust_overlapping_objects(obj_segmented_img1,obj_segmented_img2,chunk_coords,segmentation_output_path,min_overlapping_pixels_segmentation)

In [485]:
intersecting_cpls

[('10-1', '23-613'),
 ('10-2', '23-611'),
 ('10-3', '23-612'),
 ('10-4', '23-614'),
 ('10-5', '23-618'),
 ('10-6', '23-615'),
 ('10-7', '23-617'),
 ('10-8', '23-616'),
 ('10-9', '23-620'),
 ('10-10', '23-619'),
 ('10-11', '23-622'),
 ('10-13', '23-623'),
 ('10-14', '23-624'),
 ('10-15', '23-621'),
 ('10-16', '23-625'),
 ('10-17', '23-626'),
 ('10-18', '23-627'),
 ('10-19', '23-628'),
 ('10-20', '23-629'),
 ('10-21', '23-630'),
 ('10-22', '23-631'),
 ('10-23', '23-632'),
 ('10-24', '23-634'),
 ('10-25', '23-633'),
 ('10-26', '23-636'),
 ('10-27', '23-638'),
 ('10-28', '23-639'),
 ('10-29', '23-635'),
 ('10-30', '23-637'),
 ('10-31', '23-640'),
 ('10-32', '23-642'),
 ('10-33', '23-638'),
 ('10-33', '23-646'),
 ('10-34', '23-644'),
 ('10-35', '23-645'),
 ('10-37', '23-641'),
 ('10-38', '23-643'),
 ('10-39', '23-648'),
 ('10-41', '23-647'),
 ('10-42', '23-649')]

In [ ]:
intersecting_cpls_tpl = [('121-34', '174-2019'),
 ('121-34', '174-2022'),
 ('121-35', '174-2024'),
 ('121-35', '174-2025'),
 ('121-36', '174-2020'),
 ('121-36', '174-2027'),
 ('121-38', '174-2023'),
 ('121-39', '174-2024')]

In [479]:

intersecting_cpls_tpl = [tuple(x) for x in intersecting_cpls]
combined_intersecting_cpls_tpl = []
tracker = []
reducer = intersecting_cpls_tpl.copy()
for tpl in intersecting_cpls_tpl:
    print(tpl)
    if tpl not in tracker:
        combined = [tpl]
        tracker.append(tpl)
        reducer.remove(tpl)
        runner = reducer
        for cpl in runner:
            if set(tpl).intersection(cpl):
                tracker.append(cpl)
                combined.append(cpl)
                reducer.remove(cpl)
        print(combined)
        combined_intersecting_cpls_tpl.append(combined)
    print('-------')

('10-1', '23-613')
[('10-1', '23-613')]
-------
('10-2', '23-611')
[('10-2', '23-611')]
-------
('10-3', '23-612')
[('10-3', '23-612')]
-------
('10-4', '23-614')
[('10-4', '23-614')]
-------
('10-5', '23-618')
[('10-5', '23-618')]
-------
('10-6', '23-615')
[('10-6', '23-615')]
-------
('10-7', '23-617')
[('10-7', '23-617')]
-------
('10-8', '23-616')
[('10-8', '23-616')]
-------
('10-9', '23-620')
[('10-9', '23-620')]
-------
('10-10', '23-619')
[('10-10', '23-619')]
-------
('10-11', '23-622')
[('10-11', '23-622')]
-------
('10-13', '23-623')
[('10-13', '23-623')]
-------
('10-14', '23-624')
[('10-14', '23-624')]
-------
('10-15', '23-621')
[('10-15', '23-621')]
-------
('10-16', '23-625')
[('10-16', '23-625')]
-------
('10-17', '23-626')
[('10-17', '23-626')]
-------
('10-18', '23-627')
[('10-18', '23-627')]
-------
('10-19', '23-628')
[('10-19', '23-628')]
-------
('10-20', '23-629')
[('10-20', '23-629')]
-------
('10-21', '23-630')
[('10-21', '23-630')]
-------
('10-22', '23-631'

In [453]:
combined_intersecting_cpls_tpl

[[('121-34', '174-2019'), ('121-34', '174-2022')],
 [('121-35', '174-2024'), ('121-35', '174-2025'), ('121-39', '174-2024')],
 [('121-36', '174-2020'), ('121-36', '174-2027')],
 [('121-38', '174-2023')]]

In [434]:
#combined_intersecting_cpls_tpl = set(combined_intersecting_cpls_tpl)
for grp in combined_intersecting_cpls_tpl:
    if len(grp) >1 :
        # Select the object with largest overlap (maybe not best criteria)
        selected = { your_key: overlapping_sizes[your_key] for your_key in grp }
        counts = list(selected.values())
        keys = list(selected.keys())
        selected_cpl = keys[counts.index(max(counts))]
        intersecting_cpls_tpl_trimmed.append(selected_cpl)
    else:
        intersecting_cpls_tpl_trimmed.append(grp)

In [437]:
combined_intersecting_cpls_tpl

[{('121-34', '174-2019'), ('121-34', '174-2022')},
 {('121-34', '174-2019'), ('121-34', '174-2022')},
 {('121-35', '174-2024'), ('121-35', '174-2025'), ('121-39', '174-2024')},
 {('121-35', '174-2024'), ('121-35', '174-2025')},
 {('121-36', '174-2020'), ('121-36', '174-2027')},
 {('121-36', '174-2020'), ('121-36', '174-2027')},
 {('121-38', '174-2023')},
 {('121-35', '174-2024'), ('121-39', '174-2024')}]

In [420]:
counts = list(selected.values())
keys = list(selected.keys())
selected_cpl = keys[counts.index(max(counts))]

In [380]:
intersecting_cpls_removed_overlapping = intersecting_cpls_tpl.copy()
tracker = intersecting_cpls_tpl.copy()

for cpl in intersecting_cpls_tpl:
    print(cpl)
    # must check because I remove cpls and I do not alter intersecting_cpls_tpl
    if cpl in tracker:
        tracker.remove(cpl)
        print(tracker)
        runner = tracker
        for comparison_cpl in runner:
            if set(cpl).intersection(comparison_cpl) :
                tracker.remove(comparison_cpl)
                if overlapping_sizes[cpl] >= overlapping_sizes[comparison_cpl]:
                    intersecting_cpls_removed_overlapping.remove(comparison_cpl)
                else:
                    intersecting_cpls_removed_overlapping.remove(cpl)
    print('-------')

[('121-1', '174-1986'),
 ('121-2', '174-1984'),
 ('121-3', '174-1981'),
 ('121-4', '174-1985'),
 ('121-5', '174-1988'),
 ('121-6', '174-1983'),
 ('121-8', '174-1980'),
 ('121-9', '174-1989'),
 ('121-10', '174-1991'),
 ('121-11', '174-1992'),
 ('121-12', '174-1995'),
 ('121-13', '174-1996'),
 ('121-14', '174-1999'),
 ('121-15', '174-1997'),
 ('121-16', '174-1998'),
 ('121-17', '174-2001'),
 ('121-18', '174-2004'),
 ('121-19', '174-2000'),
 ('121-20', '174-2008'),
 ('121-21', '174-2010'),
 ('121-22', '174-2002'),
 ('121-23', '174-2011'),
 ('121-24', '174-2005'),
 ('121-24', '174-2017'),
 ('121-25', '174-2006'),
 ('121-26', '174-2007'),
 ('121-26', '174-2014'),
 ('121-28', '174-2009'),
 ('121-29', '174-2012'),
 ('121-30', '174-2011'),
 ('121-30', '174-2020'),
 ('121-31', '174-2014'),
 ('121-31', '174-2029'),
 ('121-32', '174-2015'),
 ('121-32', '174-2016'),
 ('121-33', '174-2016'),
 ('121-33', '174-2035'),
 ('121-34', '174-2019'),
 ('121-34', '174-2022'),
 ('121-35', '174-2024'),
 ('121-3

In [364]:
for cpl in intersecting_cpls_tpl:
    # must check because I remove cpls and I do not alter intersecting_cpls_tpl
    if cpl in tracker:
        tracker = tracker.remove(cpl)
        runner = tracker
        for comparison_cpl in runner:
            if set(cpl).intersection(comparison_cpl) :
                tracker.remove(comparison_cpl)
                if overlapping_sizes[cpl] >= overlapping_sizes[comparison_cpl]:
                    del overlapping_sizes[comparison_cpl]
                    intersecting_cpls_removed_overlapping.remove(comparison_cpl)
                else:
                    del overlapping_sizes[cpl]
                    intersecting_cpls_removed_overlapping.remove(cpl)
   

[1, 12]

In [ ]:
intersecting_cpls, objs, obj_segmented_img1, obj_segmented_img2 = adjust_overlapping_objects(obj_segmented_img1,obj_segmented_img2,chunk_coords,segmentation_output_path,min_overlapping_pixels_segmentation)
    all_obj_segmented[cpl[0]] = obj_segmented_img1
    all_obj_segmented[cpl[1]] = obj_segmented_img2
    output_cpl[cpl] = (intersecting_cpls, objs)
    corner_cpl.append(cpl[1])

In [309]:
 # Check if processing fov has one single neighbour
    if len(overlapping_dict.keys()) == 2:
        # Correct overlapping objects at the corner
        intersecting_cpls, objs, obj_segmented_img1, obj_segmented_img2 = adjust_overlapping_objects(all_obj_segmented[corner_cpl[0]],all_obj_segmented[corner_cpl[1]],chunk_coords,segmentation_output_path,min_overlapping_pixels_segmentation)
        all_obj_segmented[corner_cpl[0]] = obj_segmented_img1
        all_obj_segmented[corner_cpl[1]] = obj_segmented_img2
        output_cpl[tuple(corner_cpl)] = (intersecting_cpls, objs)
    
    return output_cpl

19

In [322]:
vw = napari.Viewer()
_ = vw.add_shapes(obj_segmented_img1['102-27'], shape_type='polygon', edge_width=5, name='obj1', face_color='magenta',visible=False)
_ = vw.add_shapes(obj_segmented_img2['141-829'], shape_type='polygon', edge_width=5, name='obj1-1', face_color='magenta',visible=False)
#_ = vw.add_shapes(obj_segmented_img1['36-15'], shape_type='polygon', edge_width=5, name='merged', face_color='magenta',visible=False)

KeyError: '141-829'

In [194]:
comparing = [tuple(x) for x in intersecting_cpls]
checked_tpls = []
out= []
cip = []
for idx,tpl in enumerate(intersecting_cpls):
    if list(tpl) not in cip:
        restart = comparing.copy()
        new_tpl = list(tpl)
        for tpl_comp in restart:
            if list(tpl) !=tpl_comp:
                if set(tpl).intersection(tpl_comp):
                    new_tpl = new_tpl + list(set(tpl).union(tpl_comp))
                    comparing.remove(tpl_comp)
                    cip.append(tpl_comp)
        #comparing.remove(list(tpl))
        out.append(new_tpl)
    

In [181]:
tpl = ('36-91', '37-90')
tpl_comp = ('36-93', '37-90')
list(set(tpl).union(tpl_comp))

['36-93', '36-91', '37-90']

In [261]:
min_overlapping_pixels_segmentation = 20
objs_list_img1 = objects_overlapping_region(obj_segmented_img1,chunk_coords)
objs_list_img2 = objects_overlapping_region(obj_segmented_img2,chunk_coords)
intersection = []
all_combinations = list(itertools.product(objs_list_img1,objs_list_img2))
for couple in all_combinations:
    obj1_set = set(map(tuple, obj_segmented_img1[couple[0]].astype(int)))
    obj2_set = set(map(tuple, obj_segmented_img2[couple[1]].astype(int)))
    result=not obj1_set.isdisjoint(obj2_set)
    if result:
        number_overlapping_pixels = len(obj1_set.intersection(obj2_set))
        if number_overlapping_pixels >= min_overlapping_pixels_segmentation:
            intersection.append(result)
        else:
            intersection.append(not result)
    else:
        intersection.append(result)
    

In [255]:
couple = ('36-93', '37-90')
inter = set(map(tuple, obj_segmented_img1[couple[0]].astype(int))).intersection(map(tuple, obj_segmented_img2[couple[1]].astype(int)))

In [256]:
len(inter)

2

In [220]:
np.array(all_combinations)[intersection]

array([['36-75', '37-73'],
       ['36-91', '37-90'],
       ['36-93', '37-89'],
       ['36-93', '37-90'],
       ['36-109', '37-106'],
       ['36-114', '37-109'],
       ['36-118', '37-110'],
       ['36-150', '37-140'],
       ['36-169', '37-152'],
       ['36-178', '37-159'],
       ['36-215', '37-186'],
       ['36-238', '37-222'],
       ['36-264', '37-242'],
       ['36-283', '37-266'],
       ['36-319', '37-291'],
       ['36-321', '37-295'],
       ['36-336', '37-321'],
       ['36-344', '37-333'],
       ['36-355', '37-339'],
       ['36-360', '37-345'],
       ['36-366', '37-349'],
       ['36-384', '37-364'],
       ['36-391', '37-369'],
       ['36-404', '37-382'],
       ['36-407', '37-394'],
       ['36-417', '37-405'],
       ['36-447', '37-442'],
       ['36-449', '37-449'],
       ['36-508', '37-509']], dtype='<U6')

In [228]:
intersection[869]

True

In [248]:
couple = ('36-508','37-509')

In [249]:
set(map(tuple, obj_segmented_img1[couple[0]].astype(int))).isdisjoint(map(tuple, obj_segmented_img2[couple[1]].astype(int)))

False

In [230]:
a = set(map(tuple, obj_segmented_img1[couple[0]].astype(int)))
b = set(map(tuple, obj_segmented_img2[couple[1]].astype(int)))

In [233]:
a.isdisjoint(b)

True

In [42]:
all_futures = []
for cpl, chunk_coords in unfolded_overlapping_regions_dict.items():
    future = client.submit(adjust_overlapping_objects,cpl,chunk_coords,segmentation_output_path)
    all_futures.append(future)
output_removal = client.gather(all_futures)

In [43]:
# tmp_saving
pickle.dump(output_removal,open(segmentation_output_path / ('output_removal_tmp.pkl'), 'wb'))

In [92]:
output_removal = pickle.load(open(segmentation_output_path / ('output_removal_tmp.pkl'), 'rb'))

In [93]:
objs_id_to_remove = []
objs_id_to_add = []
for intersecting_cpls, objs in output_removal:
    if len(intersecting_cpls):
        objs_id_to_remove.append(intersecting_cpls.flatten())
        objs_id_to_add.append(objs)

In [94]:
objs_id_to_remove = np.concatenate(objs_id_to_remove,axis=0)

In [96]:
objs_to_add_comb = {k: v for d in objs_id_to_add for k, v in d.items()}

In [101]:
objs_id_to_remove.shape[0]/2

17560.0

In [95]:
removed_dot_dict = {}
for key, items in objs_id_to_add.items():
    if key[1] not in objs_id_to_add.keys():
        removed_dot_dict[key[1]] = []
    removed_dot_dict[key[1]].append(items)

for key, items in removed_dot_dict.items():
    removed_dot_dict[key] = [el for tg in items for el in tg]

{'1-4': array([[27522, 31034],
        [27522, 31035],
        [27522, 31036],
        ...,
        [27547, 31042],
        [27547, 31043],
        [27547, 31044]])}

In [49]:
to_remove_comb = {k: v for d in to_remove for k, v in d.items()}

    removed_dot_dict = {}
    for key, items in to_remove_comb.items():
        if key[1] not in removed_dot_dict.keys():
            removed_dot_dict[key[1]] = []
        removed_dot_dict[key[1]].append(items)
    
    for key, items in removed_dot_dict.items():
        removed_dot_dict[key] = [el for tg in items for el in tg]

    for fov,dots_id_to_remove in removed_dot_dict.items():
        future = client.submit(clean_from_duplicated_dots,
                                fov = fov,
                                dots_id_to_remove=dots_id_to_remove,
                                experiment_fpath=experiment_fpath,
                                tag_cleaned_file=stitching_selected)

        all_futures.append(future)

(array([['103-20', '104-30'],
        ['103-44', '104-40'],
        ['103-44', '104-51'],
        ['103-67', '104-87'],
        ['103-67', '104-93'],
        ['103-73', '104-89'],
        ['103-78', '104-98'],
        ['103-99', '104-123'],
        ['103-139', '104-175'],
        ['103-162', '104-194'],
        ['103-171', '104-202'],
        ['103-178', '104-215'],
        ['103-205', '104-240'],
        ['103-205', '104-246'],
        ['103-227', '104-269'],
        ['103-247', '104-284'],
        ['103-267', '104-299'],
        ['103-267', '104-303'],
        ['103-279', '104-310'],
        ['103-289', '104-318'],
        ['103-318', '104-341'],
        ['103-347', '104-365'],
        ['103-369', '104-382'],
        ['103-374', '104-389'],
        ['103-389', '104-395'],
        ['103-389', '104-408'],
        ['103-416', '104-420'],
        ['103-424', '104-424'],
        ['103-456', '104-465']], dtype='<U7'),
 {'103-20': array([[15669, 38890],
         [15669, 38891],
         [15

In [45]:
intersecting_cpls, objs = adjust_overlapping_objects(cpl,chunk_coords,segmentation_output_path)

In [54]:
output_removal[36][0].flatten()

array(['19-9', '20-15', '19-70', '20-63', '19-73', '20-64', '19-100',
       '20-108', '19-104', '20-121'], dtype='<U6')

In [17]:
segmented_regions_1_viz = {el:coords_dict['stitched_coords'] for (el,coords_dict) in segmented_regions_1_dict.items()}
segmented_regions_2_viz = {el:coords_dict['stitched_coords'] for (el,coords_dict) in segmented_regions_2_dict.items()}

TotalDataDict = {**segmented_regions_1_viz, **segmented_regions_2_viz}

In [51]:
vw = napari.Viewer()
t_fov_1_r = stitching_parameters[fov_1,0]
t_fov_1_c = (stitching_parameters[fov_1,1] - metadata['img_width'])
t_fov_2_r = stitching_parameters[fov_2,0]
t_fov_2_c = (stitching_parameters[fov_2,1] - metadata['img_width'])

_ = vw.add_labels(segmented_output_1, name=str(fov_1) ,opacity=0.6, translate=(t_fov_1_r,t_fov_1_c))
_ = vw.add_labels(segmented_output_2, name=str(fov_2) ,opacity=0.6, translate=(t_fov_2_r,t_fov_2_c))
#_ = vw.add_labels(segmented_output_2, name='output_2', opacity=0.6,translate=stitching_parameters[fov_2,:])
#_ = vw.add_points((chunk[0],chunk[2]),symbol='+',size=20,name='tl',face_color='magenta')
#_ = vw.add_points((chunk[1],chunk[3]),symbol='+',size=20,name='br',face_color='cyan')


In [18]:
chunk_coords = nuclei_org_tiles.overlapping_regions[fov_1][(fov_1,fov_2)]

In [19]:
Intersecting_cpl = OverlappingCouples(chunk_coords,TotalDataDict)

In [28]:
objects_list_seg1 = obj_in_overlapping_region(segmented_regions_1_viz,chunk_coords)
objects_list_seg2 = obj_in_overlapping_region(segmented_regions_2_viz,chunk_coords)

In [53]:
vw = napari.Viewer()
for el in objects_list_seg1:
        _ = vw.add_shapes(segmented_regions_1_viz[el], shape_type='polygon', edge_width=5, name=el, face_color='magenta',visible=False)
for el in objects_list_seg2:
        _ = vw.add_shapes(segmented_regions_2_viz[el], shape_type='polygon', edge_width=5, name=el, face_color='magenta',visible=False)
   


In [82]:
  # For all the couples of objects determine which one are overlapping 
all_combinations = list(itertools.product(objects_list_seg1,objects_list_seg2))
    
Intersection = []
for couple in all_combinations:
    Result=not set(map(tuple, segmented_regions_1_viz[couple[0]].astype(int))).isdisjoint(map(tuple, segmented_regions_2_viz[couple[1]].astype(int)))
    Intersection.append(Result)
if (np.array(Intersection)).any():
    all_combinations = np.array(all_combinations)
    intersecting_cpls = all_combinations[Intersection]

In [85]:
intersecting_cpls= overlapping_objs(segmented_regions_1_viz,segmented_regions_2_viz,chunk_coords)

In [87]:
cpl = intersecting_cpls[0]

In [108]:
obj1 = segmented_regions_1_viz[cpl[0]].astype(int)
obj2 = segmented_regions_2_viz[cpl[1]].astype(int)
merged_obj = np.vstack((obj1,obj2))
merged_obj = np.unique(merged_obj,axis=0)

In [109]:
vw = napari.Viewer()
_ = vw.add_shapes(obj1, shape_type='polygon', edge_width=5, name='obj1', face_color='magenta',visible=False)
_ = vw.add_shapes(obj2, shape_type='polygon', edge_width=5, name='obj2', face_color='magenta',visible=False)
_ = vw.add_shapes(merged_obj, shape_type='polygon', edge_width=5, name='merged', face_color='magenta',visible=False)

In [114]:
objs = merge_objects(segmented_regions_1_viz,segmented_regions_2_viz,intersecting_cpls)

In [115]:
# Remove the overlpping cpls and add the new objs

{'36-75': array([[21685, 29109],
        [21685, 29110],
        [21685, 29111],
        ...,
        [21722, 29114],
        [21722, 29115],
        [21722, 29116]]),
 '36-91': array([[21728, 29165],
        [21728, 29166],
        [21728, 29167],
        ...,
        [21769, 29175],
        [21769, 29176],
        [21770, 29173]]),
 '36-93': array([[21730, 29164],
        [21730, 29165],
        [21730, 29166],
        ...,
        [21769, 29175],
        [21769, 29176],
        [21770, 29173]]),
 '36-109': array([[21784, 29121],
        [21784, 29122],
        [21784, 29123],
        ...,
        [21819, 29121],
        [21819, 29122],
        [21819, 29123]]),
 '36-114': array([[21785, 29080],
        [21785, 29081],
        [21785, 29082],
        ...,
        [21827, 29081],
        [21827, 29082],
        [21827, 29083]]),
 '36-118': array([[21794, 29144],
        [21794, 29145],
        [21794, 29146],
        ...,
        [21834, 29159],
        [21834, 29160],
        [21834,

In [94]:
obj_tuples = [tuple(row) for row in combined_obj]
obj_merged_tuples = np.unique(obj_tuples)

In [100]:
np.unique(combined_obj,axis=0).shape

(995, 2)

In [110]:
merged_obj

array([[21685, 29109],
       [21685, 29110],
       [21685, 29111],
       ...,
       [21722, 29114],
       [21722, 29115],
       [21722, 29116]])

In [34]:
all_obj_original['0-1']

array([[27306.63144851, 33558.91631974],
       [27306.63144851, 33559.91631974],
       [27307.63144851, 33558.91631974],
       [27307.63144851, 33559.91631974],
       [27308.63144851, 33557.91631974],
       [27308.63144851, 33558.91631974],
       [27308.63144851, 33559.91631974],
       [27308.63144851, 33560.91631974],
       [27308.63144851, 33561.91631974],
       [27308.63144851, 33562.91631974],
       [27309.63144851, 33555.91631974],
       [27309.63144851, 33556.91631974],
       [27309.63144851, 33557.91631974],
       [27309.63144851, 33558.91631974],
       [27309.63144851, 33559.91631974],
       [27309.63144851, 33560.91631974],
       [27309.63144851, 33561.91631974],
       [27309.63144851, 33562.91631974],
       [27309.63144851, 33563.91631974],
       [27309.63144851, 33564.91631974],
       [27309.63144851, 33565.91631974],
       [27310.63144851, 33555.91631974],
       [27310.63144851, 33556.91631974],
       [27310.63144851, 33557.91631974],
       [27310.63